# Image Classification — ViT Base on Imagenette
**TL;DR:** Compare zero-shot ViT predictions on Imagenette samples with CPU-first defaults.

**Models & Datasets:** [google/vit-base-patch16-224](https://huggingface.co/google/vit-base-patch16-224) (Apache-2.0), [Imagenette (validation split)](https://huggingface.co/datasets/frgfm/imagenette) (Apache-2.0)
**Run Profiles:** 🖥️ CPU | 🍎 Metal (Apple Silicon) | 🧪 Colab/T4 | ⚡ CUDA GPU
**Env (minimal):** python>=3.10, transformers, datasets, evaluate, accelerate (optional: peft, bitsandbytes, timm, diffusers)
**Colab:** [Open in Colab](https://colab.research.google.com/github/SSusantAchary/Hands-On-Huggingface-AI-Models/blob/main/notebooks/vision/classification-vit-base-224-imagenette_cpu-first.ipynb)

**Switches (edit in one place):**
- `device` = {"cpu","mps","cuda"}
- `precision` = {"fp32","fp16","bf16","int8","4bit"}  (apply only if supported)
- `context_len` / `image_res` / `batch_size`

**Footprint & Speed (fill after run):**
- Peak RAM: TODO
- Peak VRAM: TODO (if GPU)
- TTFB: TODO, Throughput: TODO, Load time: TODO

**Gotchas:** Disable torch.compile on MPS until PyTorch resolves precision drift ([Fixes & Tips](../fixes-and-tips/torch-compile-mps-quirks.md))



## Setup
Load a small Imagenette validation slice and prepare the ViT classifier.


In [ ]:

import json
import os
import subprocess
import time
from collections import Counter
from pathlib import Path

import pandas as pd
import torch
from datasets import load_dataset
from transformers import pipeline

from notebooks._templates.measure import append_benchmark_row, measure_memory_speed

DEVICE_PREFERENCE = os.environ.get("HF_DEVICE", "cpu")
PRECISION = os.environ.get("HF_PRECISION", "fp32")
BATCH_SIZE = int(os.environ.get("HF_BATCH", "4"))

def resolve_device(preference: str = "cpu") -> str:
    if preference == "cuda" and torch.cuda.is_available():
        return "cuda:0"
    if preference == "mps" and torch.backends.mps.is_available():
        return "mps"
    return "cpu"

DEVICE = resolve_device(DEVICE_PREFERENCE)
print(f"Using device={DEVICE}")

MODEL_ID = "google/vit-base-patch16-224"
DATASET_ID = "frgfm/imagenette"
SPLIT = "validation[:16]"
OUTPUT_DIR = Path("outputs") / "vit-imagenette"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

ds = load_dataset(DATASET_ID, split=SPLIT)
id2label = ds.features["label"].int2str
images = ds["image"]
labels = ds["label"]
true_labels = [id2label(idx) for idx in labels]


## Classify


In [ ]:

torch.manual_seed(42)

load_start = time.perf_counter()
classifier = pipeline(
    "image-classification",
    model=MODEL_ID,
    device=DEVICE,
    top_k=1,
)
load_time = time.perf_counter() - load_start

predictions = classifier(images)
pred_labels = [pred[0]["label"] if isinstance(pred, list) else pred["label"] for pred in predictions]

accuracy = sum(pl == tl for pl, tl in zip(pred_labels, true_labels)) / len(true_labels)
print(f"Accuracy on {len(true_labels)} samples: {accuracy:.3f}")

confusion = (
    pd.crosstab(pd.Series(true_labels, name="true"), pd.Series(pred_labels, name="pred"))
    .reindex(index=sorted(set(true_labels)), columns=sorted(set(pred_labels)))
    .fillna(0)
)
display(confusion)

results = pd.DataFrame(
    {
        "true_label": true_labels,
        "pred_label": pred_labels,
        "score": [pred[0]["score"] if isinstance(pred, list) else pred["score"] for pred in predictions],
    }
)
results.to_csv(OUTPUT_DIR / "predictions.csv", index=False)


## Measurement


In [ ]:

def run_inference(recorder):
    outputs = classifier(images)
    if outputs:
        recorder.mark_first_token()
    recorder.add_items(len(outputs))

metrics = measure_memory_speed(run_inference)

def fmt(value, digits=4):
    if value in (None, "", float("inf")):
        return ""
    return f"{value:.{digits}f}"

try:
    repo_commit = subprocess.check_output(["git", "rev-parse", "HEAD"], text=True).strip()
except Exception:  # noqa: BLE001
    repo_commit = ""

append_benchmark_row(
    task="imagenette-classification",
    model_id=MODEL_ID,
    dataset=DATASET_ID,
    sequence_or_image_res="224x224",
    batch=str(BATCH_SIZE),
    peak_ram_mb=fmt(metrics.get("peak_ram_mb"), 2),
    peak_vram_mb=fmt(metrics.get("peak_vram_mb"), 2),
    load_time_s=fmt(load_time, 2),
    ttfb_s=fmt(metrics.get("ttfb_s"), 3),
    tokens_per_s_or_images_per_s=fmt(metrics.get("throughput_per_s"), 3),
    precision=PRECISION,
    notebook_path="notebooks/vision/classification-vit-base-224-imagenette_cpu-first.ipynb",
    repo_commit=repo_commit,
)

with open(OUTPUT_DIR / "metrics.json", "w", encoding="utf-8") as fp:
    json.dump(metrics, fp, indent=2)
metrics


## Results Summary
        - Observations: TODO
        - Metrics captured: see `benchmarks/matrix.csv`

        ## Next Steps
        - TODOs: fill in after benchmarking

        ## Repro
        - Seed: 42 (set in measurement cell)
        - Libraries: captured via `detect_env()`
        - Notebook path: `notebooks/vision/classification-vit-base-224-imagenette_cpu-first.ipynb`
        - Latest commit: populated automatically when appending benchmarks (if git available)
